In [246]:
import numpy as np
from PIL import Image

In [361]:
image = Image.open("./balise.jpg")
image = image.resize((160,300),Image.ANTIALIAS)
data = np.asarray(image)
data.shape

def select(R, G, B):
    data_copy = data.copy()
    data_copy[:,:,2] = np.select([B], [data_copy[:,:,2]])
    data_copy[:,:,1] = np.select([G], [data_copy[:,:,1]])
    data_copy[:,:,0] = np.select([R], [data_copy[:,:,0]])
    data_copy[np.any(data_copy[:,:] == 0, axis=2)] = [0,0,0]
    return data_copy

R = np.logical_and(10 < data[:,:,0], data[:,:,0] < 35)
G = np.logical_and(80 < data[:,:,1], data[:,:,1] < 100)
B = np.logical_and(160 < data[:,:,2], data[:,:,2] < 180)

image_blue = select(R, G, B)
R = np.logical_and(40 < data[:,:,0], data[:,:,0] < 65)
G = np.logical_and(130 < data[:,:,1], data[:,:,1] < 145)
B = np.logical_and(100 < data[:,:,2], data[:,:,2] < 120)

image_green = select(R,G,B)
R = np.logical_and(180 < data[:,:,0], data[:,:,0] < 190)
G = np.logical_and(60 < data[:,:,1], data[:,:,1] < 75)
B = np.logical_and(65 < data[:,:,2], data[:,:,2] < 75)

image_red = select(R,G,B)
lignes_b, colonnes_b = np.where(image_blue[:,:,0] != 0)
lignes_g, colonnes_g = np.where(image_green[:,:,0] != 0)
lignes_r, colonnes_r = np.where(image_red[:,:,0] != 0)

baliseInFront = np.average(colonnes_r) <= np.average(colonnes_b) and (np.min(lignes_b) <= np.average(lignes_r) <= np.max(lignes_b))
baliseInFront = baliseInFront and (np.min(colonnes_b) <= np.average(colonnes_g) <= np.max(colonnes_b)) and np.average(lignes_g) <= np.average(lignes_b)
baliseInFront

True